In [ ]:
from os import path
import sys

sys.path.append("../")

import utils

from torch_geometric.utils.convert import from_networkx
import torch
import os
import numpy as np
import torch
import torch.nn.functional as F
from torch_geometric.loader import DataLoader
from sklearn.metrics import f1_score, accuracy_score
from gcn_models import GIN, GATC, SAGE, Net
from bayes_opt import BayesianOptimization

In [ ]:
early_stop_thresh = 15
best_macro_f1 = -1

In [21]:
train_dataset = utils.GraphDataset(
    "../data/", "MixedShapesSmallTrain_TRAIN", True, n_quantiles=100
)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

test_dataset = utils.GraphDataset(
    "../data/", "MixedShapesSmallTrain_TEST", True, n_quantiles=100
)
test_loader = DataLoader(test_dataset, batch_size=64)


model = Net()

optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)  # , weight_decay=1e-4)
# scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer,T_0=3,T_mult=5,verbose=True)#,
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, patience=5, mode="min", cooldown=2, factor=0.8, verbose=True
)
# scheduler = torch.optim.lr_scheduler.MultiplicativeLR(optimizer,lr_lambda=lambda x:)
# )

In [22]:
def train():
    model.train()

    total_loss = 0
    for data in train_loader:
        optimizer.zero_grad()
        # out = model(data.x.to(torch.float32), data.edge_index, data.batch)[0]
        data.x = data.x.to(torch.float32)
        out = model(data)
        loss = F.nll_loss(out, data.y - 1)
        loss.backward()
        optimizer.step()
        total_loss += float(loss) * data.num_graphs
    return total_loss / len(train_loader.dataset)


@torch.no_grad()
def test(loader, return_loss=True):
    model.eval()
    y_pred = []
    y_true = []
    loss = 0
    for data in loader:
        # y_out = model(data.x.to(torch.float32), data.edge_index, data.batch)[0]
        y_out = model(data)
        y_pred.append(y_out.argmax(dim=-1))
        y_true.append(data.y - 1)
        if return_loss:
            loss += float(F.cross_entropy(y_out, data.y - 1)) * data.num_graphs
    y_pred = np.concatenate(y_pred)
    y_true = np.concatenate(y_true)
    if return_loss:
        return (
            f1_score(y_true=y_true, y_pred=y_pred, average="macro"),
            accuracy_score(y_true=y_true, y_pred=y_pred),
            loss / len(loader.dataset),
        )
    return f1_score(y_true=y_true, y_pred=y_pred, average="macro"), accuracy_score(
        y_true=y_true, y_pred=y_pred
    )

In [23]:
for epoch in range(1, 501):
    train()
    train_macro_f1, train_acc, train_loss = test(train_loader)
    test_macro_f1, test_acc, test_loss = test(test_loader, True)
    scheduler.step(train_loss)
    print(
        f"Epoch: {epoch:03d}, Train_Loss: {train_loss:02.4f},Test_Loss: {test_loss:02.4f},Train_f1: {train_macro_f1:01.4f},Test_f1: {test_macro_f1:01.4f},Train_acc: {train_acc:01.4f},Test_acc: {test_acc:01.4f}"
    )
    if test_macro_f1 > best_macro_f1:
        best_accuracy = test_macro_f1
        best_epoch = epoch
        torch.save(model.state_dict(), "../data/best_model.pth")
    elif epoch - best_epoch > early_stop_thresh:
        print(
            f"Early stopped training at epoch {epoch} best macro F1 {best_macro_f1} in {best_epoch}"
        )
        break

Epoch: 001, Train_Loss: 6910.4812,Test_Loss: 7312.6342,Train_f1: 0.0667,Test_f1: 0.0775,Train_acc: 0.2000,Test_acc: 0.2392
Epoch: 002, Train_Loss: 6647.9703,Test_Loss: 6984.2227,Train_f1: 0.0639,Test_f1: 0.0777,Train_acc: 0.1900,Test_acc: 0.2392
Epoch: 003, Train_Loss: 6293.4893,Test_Loss: 6633.8046,Train_f1: 0.0639,Test_f1: 0.0779,Train_acc: 0.1900,Test_acc: 0.2392
Epoch: 004, Train_Loss: 5938.8445,Test_Loss: 6290.2569,Train_f1: 0.0639,Test_f1: 0.0789,Train_acc: 0.1900,Test_acc: 0.2392
Epoch: 005, Train_Loss: 5692.9267,Test_Loss: 6006.1441,Train_f1: 0.0644,Test_f1: 0.0788,Train_acc: 0.1900,Test_acc: 0.2388
Epoch: 006, Train_Loss: 5377.6299,Test_Loss: 5718.1523,Train_f1: 0.0650,Test_f1: 0.0789,Train_acc: 0.1900,Test_acc: 0.2384
Epoch: 007, Train_Loss: 5108.4401,Test_Loss: 5428.8909,Train_f1: 0.0650,Test_f1: 0.0787,Train_acc: 0.1900,Test_acc: 0.2379
Epoch: 008, Train_Loss: 4874.1461,Test_Loss: 5155.5032,Train_f1: 0.0644,Test_f1: 0.0799,Train_acc: 0.1900,Test_acc: 0.2388
Epoch: 009, Trai

In [ ]:
model = SAGE(1, [2, 4, 8, 16, 32, 64, 64, 64, 32, 16, 16, 8], aggr_method="max")
# self,
# in_channels,
# sizes,
# out_channels=5,
# pool_method="max",
# aggr_method="mean",
# normalize=True,

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, patience=5, mode="min"
)

In [ ]:
def train():
    model.train()

    total_loss = 0
    for data in train_loader:
        optimizer.zero_grad()
        out = model(data.x.to(torch.float32), data.edge_index, data.batch)
        loss = F.cross_entropy(out, data.y - 1)
        loss.backward()
        optimizer.step()
        total_loss += float(loss) * data.num_graphs
    return total_loss / len(train_loader.dataset)


@torch.no_grad()
def test(loader, return_loss=True):
    model.eval()
    y_pred = []
    y_true = []
    loss = 0
    for data in loader:
        y_out = model(
            data.x.to(torch.float32), data.edge_index, data.edge_weight, data.batch
        )
        y_pred.append(y_out.argmax(dim=-1))
        y_true.append(data.y - 1)
        if return_loss:
            loss += float(F.cross_entropy(y_out, data.y - 1)) * data.num_graphs
    y_pred = np.concatenate(y_pred)
    y_true = np.concatenate(y_true)
    if return_loss:
        return f1_score(y_true=y_true, y_pred=y_pred, average="macro"), loss / len(
            loader.dataset
        )
    return f1_score(y_true=y_true, y_pred=y_pred, average="macro")

In [ ]:
for epoch in range(1, 501):
    train()
    train_macro_f1, train_loss = test(train_loader)
    test_macro_f1, test_loss = test(test_loader, True)
    scheduler.step(train_loss)
    print(
        f"Epoch: {epoch:03d}, Train_Loss: {train_loss:02.4f}, Train: {train_macro_f1:01.4f}, Test_Loss: {test_loss:02.4f}, Test: {test_macro_f1:01.4f}"
    )
    if test_macro_f1 > best_macro_f1:
        best_accuracy = test_macro_f1
        best_epoch = epoch
        torch.save(model.state_dict(), "../data/best_model.pth")
    elif epoch - best_epoch > early_stop_thresh:
        print(
            f"Early stopped training at epoch {epoch} best macro F1 {best_macro_f1} in {best_epoch}"
        )
        break

In [ ]:
for data in train_loader:
    break

In [ ]:
data